In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

In [ ]:
class DataLoader:
  @staticmethod
  def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
    return x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
class DataPreprocessor:
  @staticmethod
  def preprocess_data(x_train, x_val, x_test):
    x_train = x_train.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
    x_val = x_val.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
    x_test = x_test.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
    return x_train, x_val, x_test

In [ ]:
class ModelBuilder:
  @staticmethod
  def build_model(input_shape, num_classes):
    model = models.Sequential()

    # Entry Flow
    model.add(layers.Conv2D(32, (3, 3), strides=(2, 2), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.DepthwiseConv2D((3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(64, (1, 1), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (1, 1), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalAveragePooling2D())

    # Fully Connected Layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [ ]:
class Trainer:
  def __init__(self, model, optimizer, loss_function, metrics):
    self.model = model
    self.model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

  def train_model(self, x_train, y_train, x_val, y_val, epochs, batch_size):
    history = self.model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val), verbose=2)
    return history

In [ ]:
def main():
  x_train, y_train, x_val, y_val, x_test, y_test = DataLoader.load_data()
  x_train, x_val, x_test = DataPreprocessor.preprocess_data(x_train, x_val, x_test)

  input_shape = x_train.shape[1:]
  num_classes = 10

  model = ModelBuilder.build_model(input_shape, num_classes)

  optimizer ='adam'
  loss_function = 'sparse_categorical_crossentropy'
  metrics = ['accuracy']

  trainer = Trainer(model, optimizer, loss_function, metrics)

  epochs = 5
  batch_size = 32

  history = trainer.train_model(x_train, y_train, x_val, y_val, epochs, batch_size)

  test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
  print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
if __name__ == '__main__':
  main()

Epoch 1/5
1500/1500 - 31s - loss: 0.4680 - accuracy: 0.8779 - val_loss: 0.4086 - val_accuracy: 0.8714 - 31s/epoch - 21ms/step
Epoch 2/5
1500/1500 - 30s - loss: 0.1389 - accuracy: 0.9599 - val_loss: 0.2137 - val_accuracy: 0.9355 - 30s/epoch - 20ms/step
Epoch 3/5
1500/1500 - 29s - loss: 0.1096 - accuracy: 0.9672 - val_loss: 0.1260 - val_accuracy: 0.9631 - 29s/epoch - 19ms/step
Epoch 4/5
1500/1500 - 29s - loss: 0.0905 - accuracy: 0.9730 - val_loss: 0.1694 - val_accuracy: 0.9505 - 29s/epoch - 19ms/step
Epoch 5/5
1500/1500 - 29s - loss: 0.0810 - accuracy: 0.9750 - val_loss: 0.2264 - val_accuracy: 0.9297 - 29s/epoch - 20ms/step
313/313 - 2s - loss: 0.2308 - accuracy: 0.9292 - 2s/epoch - 6ms/step
Test Accuracy: 92.92%
